# Prepare atlas

In [2]:
!3dresample -master cluster11_Insula_R.nii -input AAL_brain.nii.gz -prefix AAL_brain_matchedCluster.nii.gz -rmode NN

# Cluster location

Load the atlas image and information.

In [16]:
import pandas as pd
import nibabel as nib

atlasData = nib.load("AAL_brain_matchedCluster.nii.gz").get_fdata()
atlasInfo = pd.read_csv("AAL_brain.csv", header=0, index_col=0)
atlasInfo.head()

,OldIndex,Name
Index,,
1,2001,Precentral_L
2,2002,Precentral_R
3,2101,Frontal_Sup_L
4,2102,Frontal_Sup_R
5,2111,Frontal_Sup_Orb_L


Load the cluster image.

In [20]:
import numpy as np

clsImg = nib.load("cluster11_Insula_R.nii")
clsData = clsImg.get_fdata()
clsDataMask = np.copy(clsData)
clsDataMask[clsDataMask != 0] = 1

resTab = pd.DataFrame()
for i in atlasInfo.index.values:
    # mask of roi
    tmpRoiData = np.zeros(shape=atlasData.shape)
    tmpRoiData[atlasData == i] = 1
    
    tmpOverlaped = clsDataMask * tmpRoiData
    tmpNumVoxel = np.nansum(tmpOverlaped[:])
    tmpRate = tmpNumVoxel / np.nansum(tmpRoiData[:])
    
    # save results
    resTab = pd.concat([resTab, pd.DataFrame({"ROIINDEX": [i], "ROINAME": [atlasInfo.loc[i, "Name"]], "OVERLAP_NUMVOXEL": [tmpNumVoxel], "OVERLAP_RATE": [tmpRate]})])

# sort by number of voxel in overlap
resTab = resTab.sort_values(by=["OVERLAP_NUMVOXEL"], ascending=False)
resTab = resTab[resTab.OVERLAP_NUMVOXEL != 0]
resTab.to_csv("ReportsWithAAL_cluster11_Insula_R.csv", index=False)

Locating the peak.

In [25]:
import nibabel as nib
import numpy as np

clsImg = nib.load("cluster11_Insula_R.nii")
clsData = clsImg.get_fdata()
clsDataAbs = np.abs(clsData)
maxIndex = np.where(clsDataAbs == clsDataAbs.max())
maxIndex

(array([27]), array([79]), array([44]))

In [34]:
np.dot(clsImg.affine, np.reshape(np.array([24, 81, 46, 1]), [4, 1]))

array([[48. ],
       [ 1.5],
       [-3. ],
       [ 1. ]])